# Gender Prediction from Credit Card Transactions

Exploring the ways customers’ use their credit cards will give banks, researchers, marketers
and economists more information to understand their spending habits. With this kind of
knowledge, banks or marketing strategists could tailor their marketing and communication
programs according to customers’ consumption behaviors or patterns.
 
This problem focuses on credit card spending transactions to answer a simple question: could
you train an algorithm to predict the gender of the owner of a credit card?


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Beautiful format for float type
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## CSV to DataFrame

In [2]:
transactions = pd.read_csv('tj_05_credit_card_transaction.csv')

mcc_codes = pd.read_csv('mcc_codes.csv')

data_train = pd.read_csv('tj_05_training.csv', names = ['card_no', 'gender'])
data_train = data_train.drop_duplicates(keep = False, subset = 'card_no')

data_test = pd.read_csv('tj_05_test.csv', names = ['card_no'])

In [3]:
data_train.describe()

,card_no,gender
count,17414.000,17414.000
mean,1234000000014123.250,0.439
std,8104.257,0.496
min,1234000000000001.000,0.000
25%,1234000000007127.250,0.000
50%,1234000000014129.500,0.000
75%,1234000000021158.500,1.000
max,1234000000028172.000,1.000


## Merge Related DataFrame

In [4]:
transactions_with_mcc = transactions.join(mcc_codes.set_index('mcc'), on = 'mer_cat_code')

transactions_with_mcc.head()

,card_no,txn_date,txn_hour,txn_amount,mer_cat_code,mer_id,edited_description,combined_description,usda_description,irs_description,irs_reportable
0,1234000000009154,2016-12-07 00:00:00,22.000,1550.000,4511.000,0,"Airlines, Air Carriers ( not listed elsewhere)","Airlines, Air Carriers ( not listed elsewhere)","Airlines, Air Carriers ( not listed elsewhere)","Airlines, Air Carriers",Yes
1,1234000000017165,2016-12-07 00:00:00,22.000,250.000,6011.000,0,Financial Institutions – Manual Cash Disbursem...,Financial Institutions – Manual Cash Disbursem...,Financial Institutions – Manual Cash Disbursem...,Automated Cash Disburse,No1.6041-3(c)
2,1234000000000768,2016-12-07 00:00:00,22.000,250.000,8398.000,0,Charitable and Social Service Organizations,Charitable and Social Service Organizations,Charitable and Social Service Organizations,Charitable and Social Service Organizations - ...,No1.6041-3(p)(2)
3,1234000000018716,2016-12-07 00:00:00,22.000,50.000,5735.000,0,Record Shops,Record Shops,Record Shops,Record Stores,No1.6041-3(c)
4,1234000000016652,2016-12-07 00:00:00,22.000,50.000,5735.000,0,Record Shops,Record Shops,Record Shops,Record Stores,No1.6041-3(c)


## Transform Features

In [5]:
def cleanup_columns(df):
    drop_columns = ['mer_cat_code'
                    , 'mer_id'
                    , 'edited_description'
                    , 'combined_description'
                    , 'usda_description'
                    , 'irs_reportable']
    
    return df.drop(drop_columns, axis = 1).dropna()


def make_readable_columns(df):
    df.columns = ['card_no'
                  , 'date'
                  , 'hour'
                  , 'amount'
                  , 'merchant']
    
    return df


def simplify_hour(df):
    bins = (-1, 4, 12, 17, 21, 25)
    group_names = ['LateNight', 'Morning', 'Afternoon', 'Evening', 'EarlyNight']
    hours = pd.cut(df.hour, bins, labels=group_names)
    df.hour = hours
    
    return df


def build_day(df):
    df['day'] = pd.to_datetime(df.date).dt.dayofweek
    df['day'] = df['day'].apply(lambda day: {0: 'Sunday'
                                             , 1: 'Monday'
                                             , 2: 'Thuesday'
                                             , 3: 'Wednesday'
                                             , 4: 'Thursday'
                                             , 5: 'Friday'
                                             , 6: 'Saturnday'}[day])
    return df


def simplify_date(df):
    df['date'] = pd.to_datetime(df['date']).map(lambda x: x.strftime('%Y-%m-%d'))
    
    return df


def transform_features(df):
    df = cleanup_columns(df)
    df = make_readable_columns(df)
    df = simplify_hour(df)
    df = build_day(df)
    df = simplify_date(df)
    
    return df


clean_transactions = transform_features(transactions_with_mcc)
clean_transactions.describe()

,card_no,amount
count,882745.000,882745.000
mean,1234000000014116.500,1714.763
std,8076.299,26034.293
min,1234000000000001.000,50.000
25%,1234000000007269.000,250.000
50%,1234000000014186.000,500.000
75%,1234000000021104.000,1050.000
max,1234000000028171.000,19201000.000


## Visualizing Data

## Feature Engineering

### Amount of transaction per day

In [6]:
transaction_per_day = clean_transactions[['card_no', 'amount', 'day']].groupby(['card_no', 'day']).sum().reset_index()
transaction_per_day = transaction_per_day.pivot_table(values = 'amount'
                                                      , columns = 'day'
                                                      , index = 'card_no'
                                                      , aggfunc = np.sum
                                                      , fill_value = 0
                                                     )
transaction_per_day = transaction_per_day.apply(lambda column: column.apply(lambda x: np.log(x) if x > 0 else 0))
transaction_per_day.head()

day,Friday,Monday,Saturnday,Sunday,Thuesday,Thursday,Wednesday
card_no,,,,,,,
1234000000000001,7.346,8.117,7.346,8.389,8.691,8.716,11.588
1234000000000004,5.858,9.195,8.631,8.825,7.346,8.657,9.575
1234000000000005,6.477,6.215,4.605,0.000,0.000,0.000,0.000
1234000000000006,9.753,9.790,9.770,9.928,8.006,9.773,9.863
1234000000000010,8.256,8.756,9.185,8.331,9.111,9.127,9.687


### Amount of transaction per hour

In [7]:
transaction_per_hour = clean_transactions[['card_no', 'amount', 'hour']].groupby(['card_no', 'hour']).sum().fillna(0).reset_index()
transaction_per_hour = transaction_per_hour.pivot_table(values = 'amount'
                                                        , columns = ['hour']
                                                        , index = ['card_no']
                                                        , aggfunc = np.sum
                                                        , fill_value = 0
                                                       )
transaction_per_hour.columns = ['LateNight', 'Morning', 'Afternoon', 'Evening', 'EarlyNight']
transaction_per_hour = transaction_per_hour.apply(lambda column: column.apply(lambda x: np.log(x) if x > 0 else 0))
transaction_per_hour = transaction_per_hour.reset_index()
transaction_per_hour.head()

,card_no,LateNight,Morning,Afternoon,Evening,EarlyNight
0,1234000000000001,0.000,11.560,9.629,8.949,8.023
1,1234000000000004,0.000,9.756,9.315,9.324,8.466
2,1234000000000005,0.000,0.000,7.048,4.605,0.000
3,1234000000000006,8.161,11.229,10.436,0.000,0.000
4,1234000000000010,0.000,10.071,9.674,9.842,4.605


### Amount of transaction per merchant

In [8]:

transaction_per_merchant = clean_transactions[['card_no', 'amount', 'merchant']].groupby(['card_no', 'merchant']).count()
transaction_per_merchant_reset = transaction_per_merchant.reset_index()
transaction_per_merchant = transaction_per_merchant.apply(lambda x: x / transaction_per_merchant_reset['amount'][transaction_per_merchant_reset['card_no'] == x.name[0]].sum(), axis=1)

transaction_per_merchant = transaction_per_merchant.pivot_table(values = 'amount'
                                                            , columns = ['merchant']
                                                            , index = ['card_no']
                                                            , aggfunc = np.sum
                                                            , fill_value = 0
                                                           )

transaction_per_merchant = transaction_per_merchant.reset_index()
transaction_per_merchant.head()

merchant,card_no,"A/C, Refrigeration Repair",Accounting/Bookkeeping Services,Advertising Services,Agricultural Cooperative,Airlines,"Airlines, Air Carriers","Airports, Flying Fields",Amusement Parks/Carnivals,Antique Reproductions,...,Video Game Arcades,Video Tape Rental Stores,Vocational/Trade Schools,Watch/Jewelry Repair,Welding Repair,Wholesale Clubs,Wig and Toupee Stores,"Wires, Money Orders",Women’s Accessory and Specialty Shops,Women’s Ready-To-Wear Stores
0,1234000000000001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1234000000000004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.019
2,1234000000000005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1234000000000006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,1234000000000010,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.010,0.000


In [21]:
transactions_with_mer_id = transactions_with_mcc.pivot_table(values = 'mer_cat_code'
                                                            , columns = ['mer_id']
                                                            , index = ['card_no']
                                                            , aggfunc = 'count'
                                                            , fill_value = 0
                                                           ).reset_index()

In [25]:
transactions_with_mer_id.columns.name = None
transactions_with_mer_id.head()

,card_no,0,2,3,4,5,6,8,9,11,...,28719,28720,28721,28722,28723,28724,28725,28726,28727,28728
0,1234000000000001,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1234000000000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1234000000000003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1234000000000004,50,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1234000000000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Merge Feature

In [26]:
final_data = transaction_per_day.reset_index()
final_data = final_data.join(transaction_per_hour.set_index('card_no'), on='card_no')
final_data = final_data.join(transaction_per_merchant.set_index('card_no'), on='card_no')
final_data = final_data.join(transactions_with_mer_id.set_index('card_no'), on='card_no')
final_data = final_data.fillna(0)
            
final_data.describe()

,card_no,Friday,Monday,Saturnday,Sunday,Thuesday,Thursday,Wednesday,LateNight,Morning,...,28719,28720,28721,28722,28723,28724,28725,28726,28727,28728
count,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,...,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000,12572.000
mean,1234000000014099.500,7.930,7.830,7.746,7.854,7.852,8.014,7.864,2.578,8.871,...,0.001,0.000,0.001,0.000,0.000,0.002,5.493,0.065,0.000,0.000
std,8119.197,3.123,3.111,3.171,3.089,3.074,3.035,3.087,3.575,2.870,...,0.063,0.009,0.028,0.028,0.009,0.086,13.204,0.607,0.009,0.000
min,1234000000000001.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1234000000007120.750,7.576,7.431,7.378,7.467,7.438,7.650,7.467,0.000,8.420,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,1234000000014087.000,8.839,8.657,8.691,8.687,8.691,8.810,8.700,0.000,9.575,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,1234000000021015.750,9.793,9.661,9.655,9.665,9.649,9.761,9.680,6.215,10.485,...,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000
max,1234000000028171.000,15.368,15.803,15.249,15.099,15.700,15.265,16.778,14.120,16.035,...,7.000,1.000,2.000,2.000,1.000,7.000,260.000,40.000,1.000,0.000


## Fitting and Tuning an Classifier

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

In [28]:
data_train_with_features = data_train.join(final_data.set_index('card_no'), on = 'card_no', how='left').dropna()

# data_train_with_features = data_train_with_features.drop([''], axis = 1)

X_all = data_train_with_features.drop(['card_no', 'gender'], axis = 1)
y_all = data_train_with_features['gender']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state = 23)

In [29]:
X_all.shape

(9852, 26731)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [31]:
# Choose the type of classifier. 
clf = RandomForestClassifier(n_estimators = 300, min_samples_leaf = 5)

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [32]:
predictions = clf.predict(X_test)

print(accuracy_score(y_test, predictions))

0.65601217656


In [33]:
clf.score(X_train, y_train)

0.70803197563760945

## Validate with KFold

In [34]:
from sklearn.cross_validation import KFold


def run_kfold(clf):
    kf = KFold(X_all.shape[0], n_folds=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_kfold(clf)


C:\Users\User\AppData\Local\conda\conda\envs\tensorflow-notebook\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Fold 1 accuracy: 0.6561866125760649
Fold 2 accuracy: 0.6328600405679513
Fold 3 accuracy: 0.6263959390862944
Fold 4 accuracy: 0.6294416243654822
Fold 5 accuracy: 0.6741116751269035
Fold 6 accuracy: 0.6375634517766497
Fold 7 accuracy: 0.6619289340101523
Fold 8 accuracy: 0.6446700507614214
Fold 9 accuracy: 0.6395939086294417
Fold 10 accuracy: 0.6568527918781726
Mean Accuracy: 0.6459605028778533


## Model Version
### Save model to file

In [35]:
from sklearn.externals import joblib

version_name = input('version name: ')

joblib.dump(clf, 'model_{}.pkl'.format(version_name)) 

version name: test_many_features


['model_test_many_features.pkl']

### Load model from file

In [36]:
from sklearn.externals import joblib

version_name = input('version name: ')

clf = joblib.load('model_{}.pkl'.format(version_name)) 

version name: test_my_features


FileNotFoundError: [Errno 2] No such file or directory: 'model_test_my_features.pkl'

## Predict the Actual Test Data

In [37]:
test = data_test.join(final_data.set_index('card_no'), on = 'card_no', how = 'left').fillna(0)
predictions = clf.predict(test.drop('card_no', axis = 1))

output = pd.DataFrame({'gender': predictions })
output.to_csv('5-more.txt', index = False, header = None)
output.head()

,gender
count,4623.000
mean,0.111
std,0.314
min,0.000
25%,0.000
50%,0.000
75%,0.000
max,1.000


In [ ]:
output.describe()